In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords
esw = stopwords.words("english")
from string import punctuation

#remove "num", because "num" has the highest term frequency(45538) at the orinal file, 
#the second most frequent term only has a freq. of 3750
esw = esw + ['abstract', 'ci', 'hr','l','pubmed', 'num'] 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\D070678\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from collections import defaultdict
import datetime
import random
import math

In [5]:
####read file

#read train.docs file
train_doc_file = pd.read_csv("train.docs", encoding = 'utf-8', sep='\t', header=None)
train_doc_file.columns = ['id', 'text']

#read train.nontopic-titles.queries file
train_query_file = pd.read_csv("train.nontopic-titles.queries", encoding = 'utf-8', sep='\t', header=None)
train_query_file.columns = ['id', 'text']

#read example query file, only one query
#train_query_file = pd.read_csv("example.queries", encoding = 'utf-8', sep='\t', header=None)
#train_query_file.columns = ['id', 'text']



# Part 1: build functions for Random Leader IR method
text is not stemmed
1. preprocessing
2. build DTM, query vector
3. randomly select doc leader, and cluster the docs through cosine similarity
4. IR

In [6]:
## def preprocessing functions which fit with our dataset
# input file: train_doc_file["text"]/ train_query_file["text"]

def removeStopwords(file):
    i = 0
    for el in file:
        el = ' '.join([word for word in el.lower().split() if word not in esw])
        file[i] = el
        i += 1
def removePunctuation(file):
    i = 0
    for el in file:
        el = ' '.join([word for word in el.lower().split() if word not in punctuation])
        file[i] = el
        i += 1

def stemming(file):
    ps = PorterStemmer()
    i = 0
    for el in file:
        el = ' '.join([ps.stem(word) for word in el.lower().split() ])
        file[i] = el
        i += 1
        


In [7]:
##### build functions to generate document-term matrix

## get DTM, weighted by tfidf, the sqrt of the query vector is 1
# therefore, in the retrive phase, 
# dot product of the doc vector and query vector can be used to represent the cosine similarity

# input format: train_doc_file.text

def get_DTM_tfidf(file):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(file)
    vocabulary = list(train_count_vect.vocabulary_.keys())
    #print(vocabulary)

    tfidf_transformer = TfidfTransformer()
    
    X_train_tfidf = CountVectorizer(vocabulary = vocabulary)
    X_train_tfidf = X_train_tfidf.fit_transform(file)
    
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_tfidf)
    return X_train_tfidf


## get DTM, weighted by term frequency
def get_DTM_tf(file):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(file)
    vocabulary = list(train_count_vect.vocabulary_.keys())

    X_train_tf = CountVectorizer(vocabulary = vocabulary)
    X_train_tf = X_train_tf.fit_transform(file)
    return X_train_tf


In [8]:
###Generate query vector for each query

# get_QueryVector_tfidf helps to get the tiidf weighted query vector, the sqrt of the query vector is 1
# therefore, in the retrive phase, 
# dot product of the doc vector and query vector can be used to represent the cosine similarity

# input format: train_doc_file.text, train_query_file.text


def get_QueryVector_tfidf(queryFile, docFile):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(docFile)
    vocabulary = list(train_count_vect.vocabulary_.keys())
    
    if type(queryFile) is str:
        query_vect = []
        query = queryFile
        query = query.split()
        frequency = defaultdict(int)
        for el in vocabulary:
            if el in query:
                frequency[el]+= 1
            else:
                frequency[el] = 0
        query_vect = list(dict(frequency).values())/np.linalg.norm(list(dict(frequency).values()))
        
    else:
        tfidf_transformer = TfidfTransformer()
        
        query_vect = CountVectorizer(analyzer = "word", vocabulary = vocabulary)
        query_vect = query_vect.fit_transform(queryFile)
        
        query_vect = tfidf_transformer.fit_transform(query_vect)
    return query_vect



###Generate the query vector, weighted by term frequency
def get_QueryVector(queryFile, docFile):
    train_count_vect = CountVectorizer() #remove english stopwords
    X_train_counts = train_count_vect.fit_transform(docFile)
    vocabulary = list(train_count_vect.vocabulary_.keys())
    
    if type(queryFile) is str:
        query_vect = []
        query = queryFile
        query = query.split()
        frequency = defaultdict(int)
        for el in vocabulary:
            if el in query:
                frequency[el]+= 1
            else:
                frequency[el] = 0
        query_vect = list(dict(frequency).values())
        
    else:
        query_vect = CountVectorizer(analyzer = "word", vocabulary = vocabulary)
        query_vect = query_vect.fit_transform(queryFile)
        
    return query_vect

In [9]:
### information retrieve 
# return:
# leaderIndex: list of docLeaders' index
# doc_clustering: dictionary, key is the docLeader index, value is the list of docs_id

# input1: DTM_tfidf: tfidf weighted Document-term-matrix
# input2: preprocessed document file: train_doc_file 
# input3: docLeaders number (int)


def preclusteringByRandomLeader(doc_File, leaderNumber):
    
    train_tfidf = get_DTM_tfidf(doc_File.text)
    documents_id = list(doc_File['id'])
    leaderIndex = []
    doc_clustering = []
    
    for i in range(leaderNumber):
        leaderIndex.append(random.randint(0,train_tfidf.shape[0])) # randomly select doc leaders index
        doc_clustering.append([]) # initiate topic clustering

    for i in range(train_tfidf.shape[0]):
        sims = []
        #if i not in index:
        for el in leaderIndex:
            sims.append(np.dot(train_tfidf[i,], train_tfidf[el,].transpose())[0,0])
            #print(sims)    
            #maxSim = max(sims)
        maxsimindex = sims.index(max(sims))
        doc_clustering[maxsimindex].append(documents_id[i]) 
    
    return leaderIndex,doc_clustering 


In [10]:
### IR algorithm, return query results through the cosine similarity between 
## input:
# leaderIndex: list of docLeaders' index
# doc_clustering: dictionary, key is the docLeader index, value is the list of docs_id
# doc_File: preprocessed train.docs file
# query_File: preprocessed train.nontopic-titles.queries file

## output: IR results for each query, please check the IR_results for details

def IRqueryByLeaders(leaderIndex, doc_File, doc_clustering, query_File ):    
    ### get the similarity of query with each doc leader
    index = leaderIndex
    doc_clustering = doc_clustering
    
    train_tfidf = get_DTM_tfidf(doc_File.text)
    query_vector = get_QueryVector_tfidf(query_File.text, doc_File.text)
    
    documents_id = list(doc_File['id'])
    query_id = list(query_File['id'])
    r = []
    
    for q in range(query_vector.shape[0]):
        
        sims_leaders = []
        for el in index:
            sims_leaders.append(np.dot(query_vector[q], train_tfidf[el,].transpose())[0,0])

       
        maxsimindex = sims_leaders.index(max(sims_leaders)) # get the most similarity clustering index
        #print(doc_clustering[maxsimindex])

        sims_docs = []
        #if len(doc_clustering[maxsimindex])> 3:
            #get_sims_docs = []
        for el in doc_clustering[maxsimindex]:
            sims_docs.append(np.dot(query_vector[q], train_tfidf[documents_id.index(el),].transpose())[0,0]) 
            # get the similarty of query&docs in the most similarity clustering index
        
        IR_doc_sims = []
        IR_doc = []
        for i in range(len(doc_clustering[maxsimindex])):
            if sims_docs[i]>0:
                IR_doc_sims.append(sims_docs[i]) #get the non-zero similarity
                IR_doc.append(doc_clustering[maxsimindex][i]) #get the index of the docs with non-zero similarity  


        #IR_doc= []
        #for el in d:
            #IR_doc.append(doc_clustering[maxsimindex][el])


        #print(sims_leaders, sims_docs, d, IR_doc, IR_doc_sims)

        #print("Doc", "Similarity")
        if len(IR_doc_sims) >0 and len(IR_doc)>0:
            IR_doc_sims, IR_doc= zip(*sorted(zip(IR_doc_sims, IR_doc), reverse=True))# rank the results
        for j in range(len(IR_doc)):
            
            r.append([str(query_id[q]) ,  str(IR_doc[j]), IR_doc_sims[j]])

    return r

           
        
    

# Part 2: Get Functions Run !

# 1. preprocessing

# 2. get docs clustering for each randomly selected docLeaders
the number of docLeaders are configuarable

In [12]:
# get the leader docs index and the docs clusterings with clustering number =  int(math.sqrt(train_tfidf.shape[0]))
leaderIndex,doc_clustering =  preclusteringByRandomLeader(train_doc_file, int(math.sqrt(len(train_doc_file))))


In [13]:
# get the leader docs index and the docs clusterings with clustering number =  int(math.sqrt(train_tfidf.shape[0]))
leaderIndex_10,doc_clustering_10 =  preclusteringByRandomLeader(train_doc_file, 10)


# 3. retrieve results and rank them for each query
the docs are ranked through its aimilarity with the query

In [14]:
################ IRByRandomLeaderPreClustering result1, need to get the performance result #########################
# without being stemmed
# test the IR on the whole query file, #docLeaders = squareroot of total docs number, "-" converted to "and"
t1 =  datetime.datetime.now()
IR_results = IRqueryByLeaders(leaderIndex, train_doc_file, doc_clustering, train_query_file)

t2 =  datetime.datetime.now()
t = t2-t1
print("running time:", t )

# save result
df = pd.DataFrame(IR_results, columns = ['QUERY_ID', 'DOC_ID', 'sim_results'])
df.to_csv('IRByRandomLeaderPreClustering result1.txt', header=None, index=None, sep=' ', mode='a')
df

running time: 0:01:31.488563


,QUERY_ID,DOC_ID,sim_results
0,PLAIN-10,MED-2060,0.176564
1,PLAIN-10,MED-2348,0.097999
2,PLAIN-10,MED-900,0.091205
3,PLAIN-10,MED-901,0.090822
4,PLAIN-10,MED-2345,0.041052
5,PLAIN-10,MED-5108,0.036209
6,PLAIN-10,MED-2339,0.034586
7,PLAIN-10,MED-2344,0.029319
8,PLAIN-10,MED-2355,0.027664
9,PLAIN-100,MED-5186,0.221650


In [15]:
################ IRByRandomLeaderPreClustering result2, need to get the performance result #########################
# without being stemmed
# test the IR on the whole query file, #docLeaders = 10, "-" converted to "and"
t1 =  datetime.datetime.now()
IR_results = IRqueryByLeaders(leaderIndex_10, train_doc_file, doc_clustering_10, train_query_file)
t2 =  datetime.datetime.now()
t = t2-t1
print("running time:", t )

# save result
df = pd.DataFrame(IR_results, columns = ['QUERY_ID', 'DOC_ID', 'sim_results'])
df.to_csv('IRByRandomLeaderPreClustering result2.txt', header=None, index=None, sep=' ', mode='a')
df

running time: 0:04:58.470707


,QUERY_ID,DOC_ID,sim_results
0,PLAIN-10,MED-2058,0.295657
1,PLAIN-10,MED-2475,0.284176
2,PLAIN-10,MED-3774,0.274741
3,PLAIN-10,MED-2053,0.218083
4,PLAIN-10,MED-2055,0.215615
5,PLAIN-10,MED-5062,0.213023
6,PLAIN-10,MED-4975,0.194335
7,PLAIN-10,MED-1808,0.179235
8,PLAIN-10,MED-922,0.142158
9,PLAIN-10,MED-4762,0.130944


# --------------------------------------------------------------------------------------------------------------

# Don't Run Code Below 

In [11]:
### preprocessing

train_doc_file["text"] = train_doc_file['text'].str.replace('/', ' or ')

train_doc_file["text"] = train_doc_file['text'].str.replace('-', ' and ')

train_query_file["text"] = train_query_file['text'].str.replace('/', ' or ')

train_query_file["text"] = train_query_file['text'].str.replace('-', ' and ')

### remove stopwords
removeStopwords(train_doc_file['text'])
removeStopwords(train_query_file['text'])

### stemming
#stemming(train_doc_file['text'])
#stemming(train_query_file['text'])

### remove punctuation
removePunctuation(train_doc_file['text'])
removePunctuation(train_query_file['text'])

In [101]:
### create tfidf weighted DTM for the train.docs file
#train_tfidf = get_DTM_tfidf(train_doc_file.text)
#train_tfidf

<3612x23449 sparse matrix of type '<class 'numpy.float64'>'
	with 310355 stored elements in Compressed Sparse Row format>

In [118]:
### create query vector matrix for the train.nontopic-titles.queries file 
#query_vect = get_QueryVector_tfidf(train_query_file.text, train_doc_file.text)
#query_vect

<1141x23449 sparse matrix of type '<class 'numpy.float64'>'
	with 4192 stored elements in Compressed Sparse Row format>

In [ ]:
# test, each tfidf weighted vector's length is 1
#for i in range(train_tfidf.shape[1]):
 #   vector = []
  #  x = query_vect[i]
   # for i in range(x.shape[1]):
    #    vector.append(x[0,i])
    #print(vector)
    #vector_sqrt = np.linalg.norm(vector)
    #print(vector_sqrt)
    
    

1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
